In [4]:
from pprint import pprint
import os.path as osp

import numpy as np
import pandas as pd

import baposgmcp.plot as plot_utils

results_dir = "/home/jonathon/baposgmcp_results/Driving14x14WideRoundAbout-n2-v0/klr_exp_5569"
# input file
results_file_name = "compiled_episode_results.csv"
results_file_path = osp.join(results_dir, results_file_name)
# output file
compiled_results_file_name = "compiled_results.csv"
compiled_results_file_path = osp.join(results_dir, compiled_results_file_name)

In [6]:
# Outcomes parse functions
def parse_win(row):
    return int(row["episode_outcome"] == 'WIN')

def parse_loss(row):
    return int(row["episode_outcome"] == 'LOSS')

def parse_draw(row):
    return int(row["episode_outcome"] == 'DRAW')

def parse_na(row):
    return int(row["episode_outcome"] not in ('WIN', 'LOSS', 'DRAW'))

In [7]:
# Import data
ep_df = pd.read_csv(results_file_path)
ep_df = plot_utils.add_df_coplayer_policy_id(ep_df)

# Assign groups and aggs
group_keys = [
    "policy_id",
    "coplayer_policy_id"
]

# replace num_episodes with actual number of episodes completed
ep_df["num_episodes"] = ep_df.groupby(group_keys)["num_episodes"].transform(len)

# parse episode outcomes into seperate columns
outcome_col_names = ["WIN", "LOSS", "DRAW", "NA"]
for k, fn in zip(outcome_col_names, [parse_win, parse_loss, parse_draw, parse_na]):
    ep_df[k] = ep_df.apply(fn, axis=1)

print("Columns")
print("-------")
for c in ep_df.columns:
    print(f'    "{c}",')

/tmp/ipykernel_23219/2955372323.py:2: DtypeWarning: Columns (243) have mixed types. Specify dtype option on import or set low_memory=False.
  ep_df = pd.read_csv(results_file_path)


Columns
-------
    "Unnamed: 0",
    "exp_id",
    "agent_id",
    "env_name",
    "exp_seed",
    "num_episodes",
    "time_limit",
    "episode_step_limit",
    "policy_id",
    "discount",
    "c_init",
    "c_base",
    "truncated",
    "action_selection",
    "dirichlet_alpha",
    "root_exploration_fraction",
    "known_bounds",
    "extra_particles_prop",
    "step_limit",
    "epsilon",
    "other_policy_dist",
    "meta_policy_dict",
    "num_sims",
    "action_dist_distance_0",
    "action_dist_distance_1",
    "action_dist_distance_0_0",
    "action_dist_distance_0_1",
    "action_dist_distance_0_2",
    "action_dist_distance_0_3",
    "action_dist_distance_0_4",
    "action_dist_distance_0_5",
    "action_dist_distance_0_6",
    "action_dist_distance_0_7",
    "action_dist_distance_0_8",
    "action_dist_distance_0_9",
    "action_dist_distance_0_10",
    "action_dist_distance_0_11",
    "action_dist_distance_0_12",
    "action_dist_distance_0_13",
    "action_dist_distanc

In [8]:
constants = [
    "agent_id",
    "env_name",
    "time_limit",
    "episode_step_limit",
    "discount",
    "c_init",
    "c_base",
    "truncated",
    "action_selection",
    "dirichlet_alpha",
    "root_exploration_fraction",
    "known_bounds",
    "extra_particles_prop",
    "step_limit",
    "epsilon",
    "belief_size",
    "other_policy_dist",
    "meta_policy_dict",
    "num_sims",
]
replaced = [
    # replaced by number of episodes completed
    "num_episodes",
    # removed/superseded by above
    "episode_number",
    # parsed into num_outcome_...
    "episode_outcome",
    # removed/superseded by 'episode_outcome'
    "episode_done",
]
# take first value in grouped df
first_keys = [
    "Unnamed: 0",
    "exp_id",
    "exp_seed",
]

# add to constant agg keys
constants.append("num_episodes")
# add outcomes to sum keys
sum_keys = outcome_col_names

mean_keys = [
    'search_time',
    'update_time',
    'reinvigoration_time',
    'evaluation_time',
    'policy_calls',
    'inference_time',
    'search_depth',
    'min_value',
    'max_value',
    'episode_return',
    'episode_discounted_return',
    'episode_steps',
    'episode_time'
]

assigned_keys = set(group_keys + constants + replaced + first_keys + sum_keys + mean_keys)
belief_stat_keys = [c for c in ep_df if c not in assigned_keys]
belief_stat_keys

['action_dist_distance_0',
 'action_dist_distance_1',
 'action_dist_distance_0_0',
 'action_dist_distance_0_1',
 'action_dist_distance_0_2',
 'action_dist_distance_0_3',
 'action_dist_distance_0_4',
 'action_dist_distance_0_5',
 'action_dist_distance_0_6',
 'action_dist_distance_0_7',
 'action_dist_distance_0_8',
 'action_dist_distance_0_9',
 'action_dist_distance_0_10',
 'action_dist_distance_0_11',
 'action_dist_distance_0_12',
 'action_dist_distance_0_13',
 'action_dist_distance_0_14',
 'action_dist_distance_0_15',
 'action_dist_distance_0_16',
 'action_dist_distance_0_17',
 'action_dist_distance_0_18',
 'action_dist_distance_0_19',
 'action_dist_distance_0_20',
 'action_dist_distance_0_21',
 'action_dist_distance_0_22',
 'action_dist_distance_0_23',
 'action_dist_distance_0_24',
 'action_dist_distance_0_25',
 'action_dist_distance_0_26',
 'action_dist_distance_0_27',
 'action_dist_distance_0_28',
 'action_dist_distance_0_29',
 'action_dist_distance_0_30',
 'action_dist_distance_0_3

In [10]:
# group by and then aggregate
gb = ep_df.groupby(group_keys)

agg_dict = {}
for k in first_keys:
    agg_dict[k] = pd.NamedAgg(column=k, aggfunc="min")

for k in constants:
    agg_dict[k] = pd.NamedAgg(column=k, aggfunc="first")

for k in sum_keys:
    agg_dict[f"num_{k}"] = pd.NamedAgg(column=k, aggfunc="sum")
    
for k in mean_keys:
    agg_dict[f"{k}_mean"] = pd.NamedAgg(column=k, aggfunc="mean")
    agg_dict[f"{k}_std"] = pd.NamedAgg(column=k, aggfunc="std")
    agg_dict[f"{k}_min"] = pd.NamedAgg(column=k, aggfunc="min")
    agg_dict[f"{k}_max"] = pd.NamedAgg(column=k, aggfunc="max")
    
for k in belief_stat_keys:
    agg_dict[f"{k}_mean"] = pd.NamedAgg(column=k, aggfunc="mean")
    agg_dict[f"{k}_std"] = pd.NamedAgg(column=k, aggfunc="std")
    # get count of non nan values since this varies for belief stats based on step number
    agg_dict[f"{k}_n"] = pd.NamedAgg(column=k, aggfunc="count")

    
gb_agg = gb.agg(**agg_dict)
gb_agg

Unnamed: 0  \
policy_id                                        coplayer_policy_id                                             
Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0 POMetaRollout_pucb_10                                  23601   
                                                 POMetaRollout_pucb_100                                 18401   
                                                 POMetaRollout_pucb_1000                                82001   
                                                 POMetaRollout_pucb_50                                   6001   
                                                 POMetaRollout_pucb_500                                  5001   
...                                                                                                       ...   
metabaseline                                     Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0        9600   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k1_seed0-v0       15800   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k2_seed0-v0       34200   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k3_seed0-v0       17600   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k4_seed0-v0        5200   

                                                                                                   exp_id  \
policy_id                                        coplayer_policy_id                                         
Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0 POMetaRollout_pucb_10                                280   
                                                 POMetaRollout_pucb_100                               290   
                                                 POMetaRollout_pucb_1000                              300   
                                                 POMetaRollout_pucb_50                                285   
                                                 POMetaRollout_pucb_500                               295   
...                                                                                                   ...   
metabaseline                                     Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0     250   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k1_seed0-v0     251   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k2_seed0-v0     252   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k3_seed0-v0     253   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k4_seed0-v0     254   

                                                                                                   exp_seed  \
policy_id                                        coplayer_policy_id                                           
Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0 POMetaRollout_pucb_10                                    0   
                                                 POMetaRollout_pucb_100                                   0   
                                                 POMetaRollout_pucb_1000                                  0   
                                                 POMetaRollout_pucb_50                                    0   
                                                 POMetaRollout_pucb_500                                   0   
...                                                                                                     ...   
metabaseline                                     Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0         0   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k1_seed0-v0         0   
                                                 Driving14x14WideRoundAbout-n2-v0/klr_k2_seed0-v0    

In [11]:
compiled_df = gb_agg.reset_index()
compiled_df.to_csv(compiled_results_file_path)